## Configuration for fine-tuning and vLLM inference

In [1]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()

hf_token = os.getenv('HF_TOKEN')
login(token=hf_token) # or login by terminal `huggingface-cli login`

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

###-- Critical Environment Memo --###
# CUDA: 12.9
# Driver version: 575.57.08
# torch version: 2.7.1 + cu128
###-------------------------------###

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load the model for fine-tuning
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    token=hf_token,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Configure tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# Clean up model if necessary
del model
torch.cuda.empty_cache()

import gc 
gc.collect()

10472